In [14]:
%matplotlib widget
import obspy
from obspy.signal.trigger import classic_sta_lta, recursive_sta_lta, aic_simple
from obspy.signal.trigger import plot_trigger
from obspy.signal.trigger import coincidence_trigger
from obspy.signal.cross_correlation import correlation_detector
from obspy.signal import PPSD
from obspy import read_inventory
from obspy.realtime.signal import kurtosis




from hydrophone_data_processing import load, useful_variables, plotting, signal_processing
import obspy.signal.trigger as trigger
from pprint import pprint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import plot_working
import event_processing
import loading_stream_function

import matplotlib.ticker as ticker
from datetime import datetime
from datetime import timedelta
from scipy.signal import windows, freqz

In [15]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [18]:
chan = 'GHZ'
#statlist = [ 'A01' ,'A03' , 'A05' , 'A07' , 'A09' , 'A11' , 'A13' , 'A15' , 'A17' , 'A19' ]
statlist = [ 'A01' ,'A02' , 'A03' , 'A04' , 'A05' , 'A06' , 'A07' , 'A08' , 'A09', 'A10' , 'A11', 'A13', 'A14', 'A19']#, 'A16', 'A17', 'A18', 'A19' ]
#statlist = [ 'B01' ,'B02' , 'B03' , 'B04' , 'B06' , 'B07' , 'B08' , 'B09','B10' , 'B11', 'B12', 'B13' , 'B19' , 'B20']#,'B14','B15','B16','B17','B18','B19', 'B20' ]
inv = read_inventory("/media/sbrent/Oman3/7F.OmanDP.PIC.2020127.xml",format="STATIONXML")



day = 15
hr = 15
start_time = obspy.UTCDateTime('2020-1-' + str(day) + 'T00:00:00')+hr*3600
end_time = obspy.UTCDateTime('2020-1-' + str(day) + 'T00:00:00')+(hr+1)*3600
stream = obspy.read("/media/sbrent/Oman3/PASSCAL/Main_deployment/DAYS/" +statlist[0]+ "/*.." + chan +".2020.0" +str(day))


#tr = stream.select(channel=chan)[0]
#ppsd = PPSD(tr.stats, metadata = inv)
#ppsd.add(stream)

In [17]:
for s in statlist[1:]:
    
    stream += obspy.read("/media/sbrent/Oman3/PASSCAL/Main_deployment/DAYS/" + s + "/*.." + chan + ".2020.0" + str(day))
    #tr = stream[0].copy()
    #tr.remove_response(inventory=inv)  

In [ ]:
data = stream.slice(starttime=start_time,endtime=end_time) #look into trim()
#print(data)
print(start_time, end_time)

In [ ]:
#data = data.decimate(2)
data1 = data.copy()
#stream_filt = data1.filter('highpass', corners=4, freq=80, zerophase=False)

In [ ]:
stream_filt = data.filter('highpass', corners=4, freq=80, zerophase=False)
stream_2 = stream_filt.copy()

events = coincidence_trigger("classicstalta", 5, 0.8, stream_2, 3, sta=0.02, lta=0.2)




#events = coincidence_trigger("classicstalta", 4, 0.5, stream_2, 3, sta=0.08, lta=0.5)
events = pd.DataFrame(events)

In [ ]:
event_id = 509
event_meta = events.iloc[event_id]
TIME = events.time[event_id]

In [ ]:
g_wind1 = event_processing.get_event_window_raw(data1,TIME)

In [ ]:
N = len(g_wind1[0].data)
NW = 2.5
k = 4
data = g_wind1[0].data
dt=g_wind1[0].stats.delta

In [2]:
windows.dpss(NW=2.5, Kmax = 4)

TypeError: dpss() missing 1 required positional argument: 'M'

In [3]:
2.5/g_wind1[0].stats.npts

NameError: name 'g_wind1' is not defined

In [ ]:
def pmtm(signal, dpss, axis=-1):
    '''
    Estimate the power spectral density of the input signal.
    signal: n-dimensional array of real or complex values
    dpss: the Slepian matrix
    axis:   axis along which to apply the Slepian windows. Default is the last one.
    '''
    # conversion to positive-only index
    axis_p = (axis + signal.ndim) % signal.ndim
    sig_exp_shape = list(signal.shape[:axis]) + [1] + list(signal.shape[axis:])
    tap_exp_shape = [1] * axis_p + \
        list(dpss.shape) + [1] * (signal.ndim - 1 - axis_p)
    signal_tapered = signal.reshape(
        sig_exp_shape) * dpss.reshape(tap_exp_shape)
    return np.fft.fftshift(np.mean(np.absolute(np.fft.fft(signal_tapered, axis=axis_p + 1))**2, axis=axis_p), axes=axis_p)
